In [24]:
import requests
import pandas as pd
from power.ml_ops.data import get_pv_data, clean_pv_data
from power.ml_ops.model import model_yesterday

In [2]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [3]:
data = get_pv_data()


# data loaded


In [4]:
data

,Unnamed: 0.1,Unnamed: 0,local_time,electricity,irradiance_direct,irradiance_diffuse,temperature,source
0,0,315532800000,1980-01-01 01:00:00+01:00,0.0,0.0,0.0,-1.296,data/pv_data/1980_pv.csv
1,1,315536400000,1980-01-01 02:00:00+01:00,0.0,0.0,0.0,-1.216,data/pv_data/1980_pv.csv
2,2,315540000000,1980-01-01 03:00:00+01:00,0.0,0.0,0.0,-1.005,data/pv_data/1980_pv.csv
3,3,315543600000,1980-01-01 04:00:00+01:00,0.0,0.0,0.0,-1.063,data/pv_data/1980_pv.csv
4,4,315547200000,1980-01-01 05:00:00+01:00,0.0,0.0,0.0,-1.227,data/pv_data/1980_pv.csv
...,...,...,...,...,...,...,...,...
376939,8755,1672513200000,2022-12-31 20:00:00+01:00,0.0,0.0,0.0,12.240,data/pv_data/2022_pv.csv
376940,8756,1672516800000,2022-12-31 21:00:00+01:00,0.0,0.0,0.0,11.968,data/pv_data/2022_pv.csv
376941,8757,1672520400000,2022-12-31 22:00:00+01:00,0.0,0.0,0.0,11.953,data/pv_data/2022_pv.csv
376942,8758,1672524000000,2022-12-31 23:00:00+01:00,0.0,0.0,0.0,11.993,data/pv_data/2022_pv.csv


In [5]:
data_clean = clean_pv_data(data)

# data cleaned


In [23]:
def predict_previous_value(input_date: str):
    pv_data = get_pv_data()
    pv_data_clean = clean_pv_data(pv_data)
    yesterday_baseline = model_yesterday(pv_data_clean, input_date)
    values = yesterday_baseline.get('electricity').to_list()
    return {'input_date': values}

predict_previous_value('1982-07-06 05:00:00')

# data loaded
# data cleaned


{'input_date': [0.127,
  0.227,
  0.366,
  0.469,
  0.417,
  0.343,
  0.361,
  0.456,
  0.464,
  0.329,
  0.215,
  0.146,
  0.057,
  0.011,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.012,
  0.059]}

In [ ]:
data.head()

In [ ]:
def model_yesterday(X: pd.DataFrame, input_date: str) -> pd.DataFrame:
    """
    Takes an enrgy Dataframe and a date input.
    Datfrae: clean_data
    Date format: "YEAR-MONTH-DAY HOUR:MIN:SECONDS"
    Returns a Dataframe of the former day energy production
    """
    input_timestamp = pd.Timestamp(input_date, tz='UTC')
    idx = X[X.utc_time == input_timestamp].index[0]
    if idx <= 24:
        return X.iloc[0:idx,:]
    return X.iloc[idx-24:idx,:]

model_yesterday(data_clean, "1980-01-01 12:00:00")


In [27]:
base_url = 'http://127.0.0.1:8000/'
endpoint= '/predict/previous_day'
url_ = f"{base_url}{endpoint}"
params= {'input_date':"1982-07-06 05:00:00"}

response = requests.get(url_, params=params).json()
response

{'detail': 'Not Found'}

In [ ]:
result_add = seasonal_decompose(df['value'], model='additive')
result_add.plot();

In [ ]:
parameter ={}